# Boardex Graph 

Notebook to create graphs and visualise the graphs for the required adjacency matrix

In [1]:
import pandas as pd
import numpy as np

import networkx as nx

In [2]:
boardex_company_networks_us_df = pd.read_csv("raw_data/boardex_company_network_2011_2023.csv", index_col = 0)

In [3]:
boardex_company_networks_us_df.columns

Index(['associationtype', 'boardname', 'companyname', 'directorname',
       'overlapyearstart', 'overlapyearend', 'role', 'associatedrole',
       'conncompanyorgtype', 'boardid', 'companyid', 'directorid', 'roletitle',
       'roleboardposition', 'roleedflag', 'overlapyearstart_int',
       'overlapyearend_int', 'startcompanydatestartrole',
       'startcompanydateendrole', 'conncompanydatestartrole',
       'conncompanydateendrole', 'orgtype'],
      dtype='object')

In [10]:
def get_filtered_graph_data(boardex_company_networks_us_df, year):

    filtered_data_df = boardex_company_networks_us_df[['boardid', 'companyid', 'directorid', 'overlapyearstart_int', 'overlapyearend_int' ]].drop_duplicates()
    # filtered_data_df['overlapyearend'].replace("Curr", 2024)
    # filtered_data_df['overlapyearend'] = pd.to_numeric(filtered_data_df['overlapyearend'], errors='coerce')

    # filter by year here
    graph_yearly_df = filtered_data_df[ (year >= filtered_data_df['overlapyearstart_int']) & (year <= filtered_data_df['overlapyearend_int'])]
    

    return graph_yearly_df

def create_adjacency_matrix_on_interlock_df(yearly_company_network_df):

    # Assuming you have your DataFrame boardex_company_networks_us_df already loaded
    G = nx.from_pandas_edgelist(yearly_company_network_df, 'boardid', 'companyid', edge_attr='directorid')

    # Generate Adjacency Matrix
    adj_matrix = nx.adjacency_matrix(G)

    # Get the node IDs from the graph
    node_ids = list(G.nodes())

    # Convert the adjacency matrix to a DataFrame
    adj_df = pd.DataFrame(adj_matrix.todense(), index=node_ids, columns=node_ids)

    return adj_df, adj_matrix, G

Create a dataset below for each year - as a dictionary

In [15]:
# create a list of dictionary for necessary dataframe as necessary

graph_details_lst = []

for year in range(2011,2024):

    # create dictionary for each year and get details required.
    record = dict()
    record["year"] = year


    graph_yearly_df = get_filtered_graph_data(boardex_company_networks_us_df, year)

    adj_df, adj_matrix, G = create_adjacency_matrix_on_interlock_df(graph_yearly_df)

    record["adj_matrix"] = adj_matrix
    record["adj_matrix_df"] = adj_df
    record["graph"] = G

    # get the graph centrality measures
    record["eigenvector_centrality"] = nx.eigenvector_centrality(G)

    graph_details_lst.append(record)


In [16]:
graph_details_lst

[{'year': 2011,
  'adj_matrix': <276x276 sparse array of type '<class 'numpy.int64'>'
  	with 1158 stored elements in Compressed Sparse Row format>,
  'adj_matrix_df':            401.0      24296.0    2304.0     836011.0   584.0      22105.0    \
  401.0              0          1          1          1          0          0   
  24296.0            1          0          0          1          0          0   
  2304.0             1          0          0          1          0          0   
  836011.0           1          1          1          0          0          0   
  584.0              0          0          0          0          0          1   
  ...              ...        ...        ...        ...        ...        ...   
  1295244.0          0          0          0          0          0          0   
  12242.0            0          0          0          0          0          0   
  1636098.0          0          0          0          0          0          0   
  1643723.0          0  

In [17]:
pd.DataFrame(graph_details_lst)

,year,adj_matrix,adj_matrix_df,graph,eigenvector_centrality
0,2011,"(0, 1)\t1\n (0, 2)\t1\n (0, 3)\t1\n (0, 3...",401.0 24296.0 2304.0 83...,"(401.0, 24296.0, 2304.0, 836011.0, 584.0, 2210...","{401.0: 0.060019886854059484, 24296.0: 0.17156..."
1,2012,"(0, 1)\t1\n (0, 2)\t1\n (1, 0)\t1\n (1, 2...",401.0 590430.0 836011.0 56...,"(401.0, 590430.0, 836011.0, 569.0, 480064.0, 2...","{401.0: 0.005772605815752049, 590430.0: 0.0017..."
2,2013,"(0, 1)\t1\n (0, 2)\t1\n (1, 0)\t1\n (1, 2...",401.0 590430.0 836011.0 56...,"(401.0, 590430.0, 836011.0, 569.0, 480064.0, 2...","{401.0: 0.0053611960025675065, 590430.0: 0.001..."
3,2014,"(0, 1)\t1\n (1, 0)\t1\n (1, 268)\t1\n (1,...",401.0 590430.0 569.0 48...,"(401.0, 590430.0, 569.0, 480064.0, 25598.0, 58...","{401.0: 0.00026812310041453286, 590430.0: 0.00..."
4,2015,"(0, 1)\t1\n (0, 2)\t1\n (0, 16)\t1\n (0, ...",401.0 590430.0 1985116.0 56...,"(401.0, 590430.0, 1985116.0, 569.0, 480064.0, ...","{401.0: 0.023989317357032823, 590430.0: 0.0039..."
5,2016,"(0, 1)\t1\n (0, 2)\t1\n (0, 3)\t1\n (0, 8...",401.0 2129204.0 590430.0 19...,"(401.0, 2129204.0, 590430.0, 1985116.0, 569.0,...","{401.0: 0.023730803228980888, 2129204.0: 0.065..."
6,2017,"(0, 1)\t1\n (0, 2)\t1\n (0, 3)\t1\n (0, 4...",401.0 2129204.0 590430.0 19...,"(401.0, 2129204.0, 590430.0, 1985116.0, 100408...","{401.0: 0.019109269579856474, 2129204.0: 0.061..."
7,2018,"(0, 1)\t1\n (0, 2)\t1\n (0, 6)\t1\n (0, 3...",401.0 590430.0 1985116.0 56...,"(401.0, 590430.0, 1985116.0, 569.0, 25598.0, 8...","{401.0: 0.01852697353343637, 590430.0: 0.00397..."
8,2019,"(0, 1)\t1\n (0, 2)\t1\n (0, 3)\t1\n (0, 7...",401.0 2129204.0 590430.0 19...,"(401.0, 2129204.0, 590430.0, 1985116.0, 569.0,...","{401.0: 0.012670963115667901, 2129204.0: 0.056..."
9,2020,"(0, 1)\t1\n (0, 2)\t1\n (0, 39)\t1\n (0, ...",401.0 2129204.0 1985116.0 56...,"(401.0, 2129204.0, 1985116.0, 569.0, 25598.0, ...","{401.0: 0.008020370116643484, 2129204.0: 0.020..."


In [20]:
# get centrality dataframe
dfs = []


for record in graph_details_lst:
    
    centrality_tuples = list(record["eigenvector_centrality"].items())

    df = pd.DataFrame(centrality_tuples, columns = ["company_id","eigenvector_centrality"])
    df["year"] = record["year"]

    dfs.append(df)

graph_statistics_df = pd.concat(dfs, axis = 0)


In [21]:
graph_statistics_df

,company_id,eigenvector_centrality,year
0,401.0,0.060020,2011
1,24296.0,0.171565,2011
2,2304.0,0.102798,2011
3,836011.0,0.136539,2011
4,584.0,0.003195,2011
...,...,...,...
643,3487642.0,0.004561,2023
644,3383124.0,0.006360,2023
645,3408345.0,0.006750,2023
646,3395452.0,0.004703,2023


In [22]:
graph_statistics_df.to_csv("features/graph_statistics_2011_2023.csv")

In [ ]:
# currently just graph eigen centrality, but can use other statistics in here - add the approach above for other stats, or can get graph statistics, in the appraoch above,

# just add another statistic!!


Linking: Can also link the graph to boardex and gvkey tables from raw_data to link with the patent data and annual report data from before!!

### 2. Creating linking table from company network table to Director information

Can use the above dictionary to create whatever structures you need 

Then create a centrality measure below based on the connection.

Melt the graph data for each node for each year to get a director level data

https://chatgpt.com/share/fbd72cf4-19b5-4283-8300-12d964ba1bd3

Allows you to get other scores separately that's it!!

In [31]:
# need to do this at a yearly basis? Need to work from the boardex network directly!!

# then you would break the years, this is fine - just merge with the companies above, need years for each!!

yearly_long_df_lst = []


for year in range(2011, 2023):

    # print(year)

    year_graph_df = get_filtered_graph_data(boardex_company_networks_us_df, year)

    # want the directors for each year so melt it like this!!
    temp = year_graph_df[['boardid', 'companyid', 'directorid']]

    # get the long format

    # Transform the edge list to have one company per row
    df_year_long = pd.melt(temp, id_vars=['directorid'], value_vars=['boardid', 'companyid'],
                    var_name='company_type', value_name='company_id') # make sure a different name to avoid overlap!!
    
    # Drop the 'company_type' as it's no longer needed
    df_year_long = df_year_long.drop('company_type', axis=1)

    # put the year back
    df_year_long["year"]  = year

    yearly_long_df_lst.append(df_year_long)


In [32]:
yearly_long_df_all = pd.concat(yearly_long_df_lst)

yearly_long_df_all.head()

,directorid,company_id,year
0,508389.0,401.0,2011
1,340771.0,401.0,2011
2,340771.0,401.0,2011
3,654834.0,584.0,2011
4,454331.0,584.0,2011


In [33]:
yearly_long_df_all.shape # match each director id for later

(101904, 3)

In [34]:
yearly_long_df_all.to_csv("features/company_network_2011_2023_director_link_company_year.csv")

### Director specific information merge

Merge the data to the director specific employment and education backgrounds tables.

Just do so for each year, and then get the information you need later

Seeded LDA to do the classifications and then run the following after. Yes convert to each year as linking table

But do seeded LDA as required!!

In [ ]:
# # entire code example for use later

# import pandas as pd

# # Sample data for edge list
# data_edge = {
#     'companyidA': ['Comp1', 'Comp2', 'Comp3'],
#     'directorid': ['Dir1', 'Dir2', 'Dir3'],
#     'companyidB': ['Comp4', 'Comp5', 'Comp6']
# }

# # Sample data for director occupation
# data_director = {
#     'directorid': ['Dir1', 'Dir2', 'Dir3'],
#     'occupation': ['Engineer', 'Doctor', 'Artist']
# }

# # Convert dictionaries to DataFrames
# df_edge = pd.DataFrame(data_edge)
# df_director = pd.DataFrame(data_director)

# # Transform the edge list to have one company per row
# df_long = pd.melt(df_edge, id_vars=['directorid'], value_vars=['companyidA', 'companyidB'],
#                   var_name='company_type', value_name='companyid')

# # Drop the 'company_type' as it's no longer needed
# df_long = df_long.drop('company_type', axis=1)

# # Merge the transformed edge list with the occupation table
# df_merged = pd.merge(df_long, df_director, on='directorid', how='left')

# # Display the final DataFrame
# print(df_merged)
